Import Required Libraries and Spark SQL Functions

In [0]:
import os
from pyspark.sql.functions import *

Define Gold and Silver Layer Base Paths

In [0]:
silver_base = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/silver"
gold_base   = "/Volumes/adventure_works_lakehouse/adventure_works/lakehouse/gold"

Load Gold Layer Fact and Dimension Tables

In [0]:
df_fact_sales = spark.read.format("delta").load(f"{gold_base}/fact_sales")

df_dim_date = spark.read.format("delta").load(f"{gold_base}/dim_date")

df_dim_territory = spark.read.format("delta").load(f"{gold_base}/dim_territory")

Aggregate Sales by Year and Region

In [0]:
df_agg_sales_summary = (
    df_fact_sales
    .join(df_dim_date, "DateKey")
    .join(df_dim_territory, "TerritoryKey")
    .groupBy("Year", "Region")
    .agg(
        round(sum("Revenue"), 4).alias("TotalRevenue"),
        round(sum("Profit"), 4).alias("TotalProfit"),
        sum("OrderQuantity").alias("TotalQuantity"),
        countDistinct("OrderNumber").alias("TotalOrders")
    )
)

Write Aggregated Sales Summary to Gold Layer

In [0]:
df_agg_sales_summary.write.format("delta")\
    .mode("overwrite")\
    .save(f"{gold_base}/agg_sales_summary")

Save Aggregated Sales Summary as a Gold Table

In [0]:
df_agg_sales_summary.write.format("delta")\
    .mode("overwrite")\
    .saveAsTable("adventure_works_lakehouse.adventure_works.agg_sales_summary")


In [0]:
%sql
SELECT * FROM adventure_works_lakehouse.adventure_works.agg_sales_summary

Year,Region,TotalRevenue,TotalProfit,TotalQuantity,TotalOrders
2016,Germany,1005008.7896,419351.9699,3338,974
2016,Southwest,1647370.1778,711672.9396,7287,2067
2017,Northeast,2402.575,1157.7025,24,5
2017,Central,307.7404,170.007,14,5
2016,Southeast,2945.4759,1348.1324,13,6
2017,Australia,2408104.871,995203.0102,9487,2655
2016,Northeast,3998.9928,1716.8792,16,5
2015,United Kingdom,567033.0718,231486.0544,255,255
2015,Southwest,1264207.6133,513290.2552,514,514
2015,Northwest,834500.5988,337184.2405,334,334


Databricks visualization. Run in Databricks to view.